In [1]:
!pip install jsonlines

In [34]:
import jsonlines as jl 
import pandas as pd 
import os 
import numpy as np 

In [29]:
df = pd.read_json('roam_prescription_based_prediction.jsonl' , lines=True)
df.head()

,cms_prescription_counts,provider_variables,npi
0,"{'DOXAZOSIN MESYLATE': 26, 'MIDODRINE HCL': 12...","{'settlement_type': 'non-urban', 'generic_rx_c...",1295763035
1,"{'CEPHALEXIN': 23, 'AMOXICILLIN': 52, 'HYDROCO...","{'settlement_type': 'non-urban', 'generic_rx_c...",1992715205
2,"{'CEPHALEXIN': 28, 'AMOXICILLIN': 73, 'CLINDAM...","{'settlement_type': 'non-urban', 'generic_rx_c...",1578587630
3,{'AMOXICILLIN': 63},"{'settlement_type': 'non-urban', 'generic_rx_c...",1932278405
4,"{'PIOGLITAZONE HCL': 24, 'BENAZEPRIL HCL': 29,...","{'settlement_type': 'non-urban', 'generic_rx_c...",1437366804


In [32]:
unique_drugs = set()
for dct in df['cms_prescription_counts'].values:
    for item in dct.keys():
        unique_drugs.add(item)
len(unique_drugs)

2397

In [33]:
with jl.open('roam_prescription_based_prediction.jsonl') as file_:
    for obj in file_:
        print(obj)
        break

{'cms_prescription_counts': {'DOXAZOSIN MESYLATE': 26, 'MIDODRINE HCL': 12, 'MEGESTROL ACETATE': 11, 'BENAZEPRIL HCL': 11, 'METOLAZONE': 73, 'NOVOLOG': 12, 'DIAZEPAM': 24, 'HYDRALAZINE HCL': 50, 'SENSIPAR': 94, 'LABETALOL HCL': 28, 'PREDNISONE': 40, 'CALCITRIOL': 79, 'HYDROCODONE-ACETAMINOPHEN': 64, 'HYDROCHLOROTHIAZIDE': 59, 'LOSARTAN-HYDROCHLOROTHIAZIDE': 14, 'FENOFIBRATE': 14, 'MINOXIDIL': 14, 'MELOXICAM': 29, 'ATENOLOL': 62, 'CARISOPRODOL': 40, 'GABAPENTIN': 35, 'OMEPRAZOLE': 35, 'KLOR-CON M10': 20, 'LANTUS': 20, 'AMLODIPINE BESYLATE': 175, 'CARVEDILOL': 36, 'LOSARTAN POTASSIUM': 41, 'IRBESARTAN': 11, 'NIFEDICAL XL': 32, 'NIFEDIPINE ER': 51, 'LEVOTHYROXINE SODIUM': 12, 'POTASSIUM CHLORIDE': 30, 'FUROSEMIDE': 162, 'GLYBURIDE': 16, 'CLONIDINE HCL': 43, 'TEMAZEPAM': 41, 'SPIRONOLACTONE': 50, 'LOVASTATIN': 11, 'LISINOPRIL': 44, 'PANTOPRAZOLE SODIUM': 13, 'CALCIUM ACETATE': 85, 'NEXIUM': 44, 'ZOLPIDEM TARTRATE': 41, 'DIOVAN': 20, 'OXYCODONE HCL': 51, 'METOPROLOL SUCCINATE': 104, 'RANITI

In [60]:
# Identified our product as PREDNISONE
# Market drugs as METOLAZONE,OMEPRAZOLE,LISINOPRIL
def get_prednisone_rx(drug_dct):
    if('PREDNISONE' in list(drug_dct.keys())):
        return drug_dct['PREDNISONE']
    else:
        return 0

def get_market_rx(drug_dct):
    total_rx = 0
    if('PREDNISONE' in list(drug_dct.keys())):
        total_rx += drug_dct['PREDNISONE']
    if('METOLAZONE' in list(drug_dct.keys())):
        total_rx += drug_dct['METOLAZONE']
    if('OMEPRAZOLE' in list(drug_dct.keys())):
        total_rx += drug_dct['OMEPRAZOLE']
    if('LISINOPRIL' in list(drug_dct.keys())):
        total_rx += drug_dct['LISINOPRIL']
    return total_rx

In [61]:
df['prednisone_rx'] = df['cms_prescription_counts'].apply(get_prednisone_rx)
df['market_rx'] = df['cms_prescription_counts'].apply(get_market_rx)

In [62]:
df['market_rx'].value_counts()

0       149265
12        2621
11        2616
13        2126
14        1744
         ...  
1824         1
1312         1
4325         1
1099         1
1663         1
Name: market_rx, Length: 1520, dtype: int64

In [213]:
df_pred_market = df.loc[df['market_rx'] > 0,:].copy()

In [214]:
df_pred_market.shape

(90665, 5)

In [215]:
149265+90665

239930

In [216]:
df.shape

(239930, 5)

In [217]:
df_pred_market.head()

,cms_prescription_counts,provider_variables,npi,prednisone_rx,market_rx
0,"{'DOXAZOSIN MESYLATE': 26, 'MIDODRINE HCL': 12...","{'settlement_type': 'non-urban', 'generic_rx_c...",1295763035,40,192
4,"{'PIOGLITAZONE HCL': 24, 'BENAZEPRIL HCL': 29,...","{'settlement_type': 'non-urban', 'generic_rx_c...",1437366804,0,77
8,"{'CARAFATE': 16, 'CARBIDOPA-LEVODOPA': 29, 'PR...","{'settlement_type': 'urban', 'generic_rx_count...",1316196462,61,318
12,"{'GABAPENTIN': 24, 'ALLOPURINOL': 14, 'HUMIRA'...","{'settlement_type': 'non-urban', 'generic_rx_c...",1033483755,57,57
14,"{'CARAFATE': 19, 'OMEPRAZOLE': 133, 'HUMIRA': ...","{'settlement_type': 'non-urban', 'generic_rx_c...",1518951466,0,145


In [218]:
# Create market and product deciles
df_pred_market_zero_pred = df_pred_market.loc[df_pred_market['prednisone_rx'] == 0,:].copy()
df_pred_market_nz_pred = df_pred_market.loc[df_pred_market['prednisone_rx'] > 0,:].copy()

# Create deciles for non zero prednisone_rx
df_pred_market_nz_pred['prednisone_decile'] = pd.qcut(df_pred_market_nz_pred['prednisone_rx'] , q = [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0] , labels=[1,2,3,4,5,6,7,8,9,10])

# Assign 0 zero decile 
df_pred_market_zero_pred['prednisone_decile'] = 0

# Create deciles for market_rx (all are non zero)
df_pred_market['market_decile'] = pd.qcut(df_pred_market['market_rx'] , q = [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0] , labels=[1,2,3,4,5,6,7,8,9,10] )

In [219]:
df_pred_market_zero_pred.head()

,cms_prescription_counts,provider_variables,npi,prednisone_rx,market_rx,prednisone_decile
4,"{'PIOGLITAZONE HCL': 24, 'BENAZEPRIL HCL': 29,...","{'settlement_type': 'non-urban', 'generic_rx_c...",1437366804,0,77,0
14,"{'CARAFATE': 19, 'OMEPRAZOLE': 133, 'HUMIRA': ...","{'settlement_type': 'non-urban', 'generic_rx_c...",1518951466,0,145,0
18,"{'LEVOFLOXACIN': 12, 'OMEPRAZOLE': 35, 'REVLIM...","{'settlement_type': 'non-urban', 'generic_rx_c...",1013940501,0,35,0
20,"{'CLONAZEPAM': 11, 'ENALAPRIL MALEATE': 25, 'S...","{'settlement_type': 'non-urban', 'generic_rx_c...",1275533044,0,221,0
31,"{'BENAZEPRIL HCL': 16, 'INSULIN SYRINGE': 11, ...","{'settlement_type': 'non-urban', 'generic_rx_c...",1669526109,0,136,0


In [220]:
df_pred_market_nz_pred.head()

,cms_prescription_counts,provider_variables,npi,prednisone_rx,market_rx,prednisone_decile
0,"{'DOXAZOSIN MESYLATE': 26, 'MIDODRINE HCL': 12...","{'settlement_type': 'non-urban', 'generic_rx_c...",1295763035,40,192,7
8,"{'CARAFATE': 16, 'CARBIDOPA-LEVODOPA': 29, 'PR...","{'settlement_type': 'urban', 'generic_rx_count...",1316196462,61,318,8
12,"{'GABAPENTIN': 24, 'ALLOPURINOL': 14, 'HUMIRA'...","{'settlement_type': 'non-urban', 'generic_rx_c...",1033483755,57,57,8
17,"{'OMEPRAZOLE': 11, 'TARGRETIN': 11, 'LEVOTHYRO...","{'settlement_type': 'non-urban', 'generic_rx_c...",1881850626,37,48,7
23,"{'PROVENTIL HFA': 20, 'ADVAIR DISKUS': 120, 'V...","{'settlement_type': 'urban', 'generic_rx_count...",1992021687,126,126,9


In [221]:
df_pred_market_zero_pred.isna().sum()

cms_prescription_counts    0
provider_variables         0
npi                        0
prednisone_rx              0
market_rx                  0
prednisone_decile          0
dtype: int64

In [222]:
df_pred_market_nz_pred.isna().sum()

cms_prescription_counts    0
provider_variables         0
npi                        0
prednisone_rx              0
market_rx                  0
prednisone_decile          0
dtype: int64

In [223]:
df_pred_market_nz_pred[df_pred_market_nz_pred['prednisone_decile'].isna()]['prednisone_rx'].value_counts()

Series([], Name: prednisone_rx, dtype: int64)

In [224]:
df_pred_market_nz_pred['prednisone_rx'].value_counts()

11      2280
12      2122
13      1884
14      1662
15      1614
        ... 
687        1
623        1
2286       1
1166       1
1311       1
Name: prednisone_rx, Length: 846, dtype: int64

In [225]:
df_total_pred = df_pred_market_zero_pred.append(df_pred_market_nz_pred)

In [226]:
df_total_pred.shape

(90665, 6)

In [227]:
df_total_pred.isna().sum()

cms_prescription_counts    0
provider_variables         0
npi                        0
prednisone_rx              0
market_rx                  0
prednisone_decile          0
dtype: int64

In [228]:
df_pred_market['market_rx'].value_counts()

12      2621
11      2616
13      2126
14      1744
15      1548
        ... 
1130       1
3085       1
2347       1
1292       1
1663       1
Name: market_rx, Length: 1519, dtype: int64

In [229]:
df_pred_market[df_pred_market['market_decile'].isna()]['market_rx'].value_counts()

Series([], Name: market_rx, dtype: int64)

In [230]:
df_with_decile = pd.merge(df_total_pred , df_pred_market[['npi','market_decile']] , on = 'npi' , how = 'inner')

In [231]:
df_with_decile.head()

,cms_prescription_counts,provider_variables,npi,prednisone_rx,market_rx,prednisone_decile,market_decile
0,"{'PIOGLITAZONE HCL': 24, 'BENAZEPRIL HCL': 29,...","{'settlement_type': 'non-urban', 'generic_rx_c...",1437366804,0,77,0,5
1,"{'CARAFATE': 19, 'OMEPRAZOLE': 133, 'HUMIRA': ...","{'settlement_type': 'non-urban', 'generic_rx_c...",1518951466,0,145,0,7
2,"{'LEVOFLOXACIN': 12, 'OMEPRAZOLE': 35, 'REVLIM...","{'settlement_type': 'non-urban', 'generic_rx_c...",1013940501,0,35,0,3
3,"{'CLONAZEPAM': 11, 'ENALAPRIL MALEATE': 25, 'S...","{'settlement_type': 'non-urban', 'generic_rx_c...",1275533044,0,221,0,8
4,"{'BENAZEPRIL HCL': 16, 'INSULIN SYRINGE': 11, ...","{'settlement_type': 'non-urban', 'generic_rx_c...",1669526109,0,136,0,7


In [232]:
df_with_decile.shape

(90665, 7)

In [233]:
df_with_decile.isna().sum()

cms_prescription_counts    0
provider_variables         0
npi                        0
prednisone_rx              0
market_rx                  0
prednisone_decile          0
market_decile              0
dtype: int64

In [234]:
df_pred_market_nz_pred['prednisone_decile'].value_counts()

2     5160
4     4460
1     4402
8     4011
6     4004
10    3982
9     3911
7     3887
5     3377
3     2683
Name: prednisone_decile, dtype: int64

In [235]:
df_with_decile.groupby('prednisone_decile')['prednisone_rx'].agg(['sum','count'])

,sum,count
prednisone_decile,,
0,0,50788
1,50544,4402
2,71970,5160
3,44226,2683
4,86665,4460
5,79185,3377
6,114994,4004
7,145761,3887
8,213332,4011


In [236]:
df_with_decile.groupby('market_decile')['market_rx'].agg(['sum','count'])

,sum,count
market_decile,,
1,112282,9107
2,180722,9773
3,251712,8650
4,385464,8861
5,593625,9218
6,829862,8916
7,1196758,9024
8,1709372,8996
9,2614653,9086
